In [1]:
# NOTE
# cypher.forbid_exhaustive_shortestpath=true set in neo4j conf file
# https://neo4j.com/docs/operations-manual/current/configuration/neo4j-conf/

In [2]:
from py2neo import *
import pandas as pd

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [4]:
graph = Graph()

## FUNCTIONS

In [5]:
def nodeInfo(a):
    commandToRun = 'MATCH (pages:Page {id: %s}) \
                RETURN pages' % (a)
    return graph.run(commandToRun).data()

##### Return similarity statistics for two sets (intersection, union, Jaccard coefficient)

In [6]:
# compute similarity statistics
def similarityStats(a,b):
    intSize = len(a.intersection(b))
    unionSize = len(a.union(b))
    
    if unionSize == 0:
        jaccard = 0
    else:
        jaccard = intSize / unionSize
    
    return (intSize, unionSize, jaccard)

##### Return identifying information of parent categories of chosen article or category as pandas dataframe

In [7]:
# Give wikipedia id as integer as function argument (e.g. cateogry "Finland" = 693995)
def parentCategories(a):
    wikiID = a
    commandToRun = 'MATCH (pages:Category:Page) \
                <-[:BELONGS_TO]- \
                (:Page {id: %s}) \
                RETURN pages.title, pages.id' % (wikiID)

    # ensure that a dataframe with correct columns is returnd also when query is empty
    out = pd.DataFrame(columns = ["pages.title", "pages.id"])
    out = out.append(graph.run(commandToRun).to_data_frame())
    
    return out


##### Return identifying information of children (both category and article) of chosen category as pandas dataframe

In [8]:
# Give wikipedia id as integer as function argument (e.g. cateogry "Finland" = 693995)
def childPages(a):
    wikiID = a
    commandToRun = 'MATCH (pages:Page) \
                -[:BELONGS_TO]-> \
                (:Page {id: %s}) \
                RETURN pages.title, pages.id' % (wikiID)

    # ensure that a dataframe with correct columns is returnd also when query is empty
    out = pd.DataFrame(columns = ["pages.title", "pages.id"])
    out = out.append(graph.run(commandToRun).to_data_frame())
    
    return out

##### Return dataframe with all articles (but not categories) with given title [only one result is expected]

In [9]:
# Give wikipedia title as string as function argument
def articleByTitle(a):
    ArticleToFind = a
    commandToRun = 'MATCH (articles:Page {title: "%s"}) \
                    WHERE NONE(art IN [articles] WHERE art:Category) \
                    RETURN articles.title, articles.id, ID(articles)' % (ArticleToFind)
    return graph.run(commandToRun).to_data_frame()

##### Return dataframe with all categories (but not articles) with given title [only one result is expected]

In [10]:
# Give wikipedia title as string as function argument
def categoryByTitle(a):
    CategoryToFind = a
    commandToRun = 'MATCH (categories:Category:Page {title: "%s"}) \
                    RETURN categories.title, categories.id, ID(categories)' % (CategoryToFind)
    return graph.run(commandToRun).to_data_frame()

##### Return dataframe containing shortest path between input node (article or category) and Main_topics_classification category

In [11]:
def shortestPathToMTC(a):
    # (:Page {id: 7345184}) is Main_topics_classifications category node
    inputNode = a

    commandToRun = 'MATCH path=shortestPath( \
                    (:Page {id: %s})-[:BELONGS_TO*0..10]->(:Page {id: 7345184})) \
                    UNWIND nodes(path) AS pages \
                    RETURN pages.title, pages.id, ID(pages)' % (inputNode)

    return pd.DataFrame(graph.run(commandToRun).data())

##### Return similarity value between two categories as defined by Biuk-Aghai & Cheang (2011)

In [12]:
# Take as input tuple containing depth of category to compare to as well as intersection of children between two categories

'''
Given parent category p and child category c,
and given a root category node r, we calculate the category similarity
Sp;c as: Sp;c = Dc - Cp;c / k , where Dc is the depth of category
c in the category graph, i.e. the shortest distance from the root
category node r; Cp;c is the number of co-assigned articles of categories
p and c; and k is a constant that is empirically determined.
Through experimentation we have found that a value of k = 2 produces
the best results, i.e. results that agree with human intuition as
to similarity of a given pair of categories. A smaller value of Sp;c
indicates a greater similarity (i.e. a smaller distance between the
nodes). The number of co-assigned articles Cp;c of parent category
p and child category c is simply the cardinality of the intersection
of their assigned article sets: Cp;c = jAp \ Acj, where Ap and Ac
are the sets of articles assigned to categories p and c, respectively.
'''
# Depth is calcualted for parent when going bottom-up in graph
# C is calculated using intersection of both child articles and sub-categories

def similarityBAC(a):
    d = a[0]
    c = a[1]
    k = 2
    return d - (c/k)

##### Return dataframe containing all parent categories of category a and similarity statistics to each as well as parent depth to MTC

In [13]:
# Give wikipedia id as integer as function argument (e.g. cateogry "Finland" = 693995)
def parentSimilarities(a):
    parents = parentCategories(a)
    children = childPages(a)
    
    if len(parents) == 0:
        raise ValueError("Category processed does not have parents (likely input category to chosenPathUpToMTC() if called)")
    
    # Create columns with similarity stats using functions similarityStats
    parents["similarities"] = parents["pages.id"].apply(lambda x: similarityStats(set(children["pages.id"]), set(childPages(x)["pages.id"])))
    parents[["intersection", "union", "jaccard"]] = pd.DataFrame(parents['similarities'].tolist(), index = parents.index)
    parents.drop(["similarities"], axis = 1, inplace = True)
        
    # Add column with parent category depth (steps to Main_topics_classifications node)
    parents["depth"] = parents["pages.id"].apply(lambda x: len(shortestPathToMTC(x))-1)
    
    # Add column with similarityBAC
    parents["similarityBAC-aid"] = list(zip(parents["depth"], parents["intersection"]))
    parents["similarityBAC"] = parents["similarityBAC-aid"].apply(lambda x: similarityBAC(x))
    parents.drop(["similarityBAC-aid"], axis = 1, inplace = True)
    
    # Sort ascending
    parents.sort_values(by = "similarityBAC", ascending = True, inplace = True)
    parents.reset_index(drop = True, inplace = True)
    
    return parents

##### Return node based on neo4j database ID [NOTE: not same as wikipedia ID used elsewhere]

In [14]:
def getWithNeoID(a):
    return NodeMatcher(graph).get(a)

##### Return dataframe containing info of what category to choose from parentSimilarities() output

In [15]:
'''
we choose which parent link to keep according to
following rules: (1) Choose the parent whose similarity value Sp;c
is lower; (2) If Sp1;c = Sp2;c, choose the parent whose depth D is
lower; (3) If Dp1 = Dp2, choose the parent with the larger value
of Cp;c; (4) If Cp1;c = Cp2;c, choose the parent with the lower
page ID.
'''
# Takes parentSimilarities() / or potentially child similarities output dataframe as input
def chooseCategoryPath(a):
    a.sort_values(by = ["similarityBAC", "depth"], ascending = True, inplace = True)
    a["mostSimilar"] = "False"
    a["comment"] = ""
    
    # Set value for mostSimilar to "Not connected" for rows with depth = -1 i.e. no connection to MTC
    a.loc[a["depth"] == -1, "comment"] = "Not connected"
    
    # Set value for mostSimilar to "True" for rows that are not "Not connected" and that have the minimum value of similarityBAC
    workingDF = a.loc[a["comment"] != "Not connected"]
    selectedIndexes = workingDF.loc[workingDF["similarityBAC"] == workingDF["similarityBAC"].min()].index
    
    a.loc[selectedIndexes, "mostSimilar"] = "True"
    a.loc[selectedIndexes, "comment"] = "Lowest similarityBAC"
    
    workingDF = a.loc[a["mostSimilar"] == "True"]
    
    if len(workingDF) > 1:
        # Set all mostSimilar of partial dataframe and output back to False, then set min depth rows to true
        workingDF["mostSimilar"] = "False"
        a["mostSimilar"] = "False"
        selectedIndexes = workingDF.loc[workingDF["depth"] == workingDF["depth"].min()].index
        
        a.loc[selectedIndexes, "mostSimilar"] = "True"
        a.loc[selectedIndexes, "comment"] = a.loc[selectedIndexes, "comment"] + "; Lowest depth"
        
        workingDF = a.loc[a["mostSimilar"] == "True"]
        
        # If several rows now set to true, test for highest intersection
        if len(workingDF) > 1:
            # Set all mostSimilar of partial dataframe and output back to False, then set max intersection rows to true
            workingDF["mostSimilar"] = "False"
            a["mostSimilar"] = "False"
            selectedIndexes = workingDF.loc[workingDF["intersection"] == workingDF["intersection"].max()].index
            
            a.loc[selectedIndexes, "mostSimilar"] = "True"
            a.loc[selectedIndexes, "comment"] = a.loc[selectedIndexes, "comment"] + "; Highest intersection"
            
            workingDF = a.loc[a["mostSimilar"] == "True"]
            
            # If several rows now set to true, choose row with lowes pages.id
            if len(workingDF) > 1:
                # Set all mostSimilar of partial dataframe and output back to False, then set min wikipedia id row (only one) to true
                workingDF["mostSimilar"] = "False"
                a["mostSimilar"] = "False"
                selectedIndexes = workingDF.loc[workingDF["pages.id"] == workingDF["pages.id"].min()].index
                
                a.loc[selectedIndexes, "mostSimilar"] = "True"
                a.loc[selectedIndexes, "comment"] = a.loc[selectedIndexes, "comment"] + "; Lowest wikipedia id"
    
    
    return a

##### Return dataframe containing info of chosen path to MTC (iterates chooseCategoryPath() upwards)

In [16]:
# Iterate chooseCategoryPath() from input category (wikipedia id as input) until MTC is reached. Return dataframe with chosen path rows
# Root node category "Main_topic_classifications" has pages.id = 7345184

# NOTE: Error if input category does not have parents
def chosenPathUpToMTC(a):
    mtcFound = False
    nextStep = a
    chosenPath = pd.DataFrame()
    
    while(not mtcFound):
        allParents = parentSimilarities(nextStep)
        allParents = chooseCategoryPath(allParents)
        
        # If allParents contains MTC category
        if(len(allParents.loc[allParents["pages.id"] == 7345184]) == 1):
            rowToAppend = allParents.loc[allParents["pages.id"] == 7345184]
            mtcFound = True
        else:
            rowToAppend = allParents.loc[allParents["mostSimilar"] == "True"]
            nextStep = int(allParents.loc[allParents["mostSimilar"] == "True", "pages.id"])
        
        chosenPath = chosenPath.append(rowToAppend)
        chosenPath.reset_index(drop = True, inplace = True)
    
    
    return chosenPath

##### Article strength calculations

In [17]:
# Return dataframe with all pages linking to or from input page
def linksBetween(a):
    wikiID = a
    commandToRun = 'MATCH (pages:Page) \
                -[:LINKS_TO]- \
                (:Page {id: %s}) \
                RETURN pages.title, pages.id' % (wikiID)

    # ensure that a dataframe with correct columns is returnd also when query is empty
    out = pd.DataFrame(columns = ["pages.title", "pages.id"])
    out = out.append(graph.run(commandToRun).to_data_frame())
    
    return out


In [18]:
# a as pages.id for artice, c as pages.id for parent category
def articleClassificationStrength(a, c):
    aLinks = set(linksBetween(a)["pages.id"])
    cChildren = set(childPages(c)["pages.id"])
    
    intersectionSize = len(aLinks.intersection(cChildren))
    
    return 1 + intersectionSize

In [19]:
def strongestArticleParents(a):
    parents = parentCategories(a)
    parents["depth"] = parents["pages.id"].apply(lambda x: len(shortestPathToMTC(x)) -1 )
    parents.loc[parents["depth"] != -1 , "Strength"] = parents["pages.id"].apply(lambda x: articleClassificationStrength(a, x))
    parents.sort_values(by = ["Strength"], ascending = False, inplace = True)
   
    
    return parents

In [20]:
def chosenPathArticleToMTC(a):
    strongestParent = strongestArticleParents(a)
    path = chosenPathUpToMTC(strongestParent.iloc[0,1])
    
    path.loc[-1] = strongestParent.iloc[0, :3]
    path.sort_index(inplace = True)
    path.reset_index(drop = True, inplace = True)
    
    return path

## Adding node labels and properties

#### Database edits (run commands commented out to prevent accidental runs)

In [21]:
%%time
# Add isCategory property with value 1 to all category pages

commandToRun = 'MATCH (pages:Category:Page) \
                SET pages.isCategory = 1'

#graph.run(commandToRun)

Wall time: 0 ns


In [22]:
%%time
# Add isCategory property with value 0 to all article pages

commandToRun = 'MATCH (pages:Page) \
                WHERE NONE(art IN [pages] WHERE art:Category) \
                SET pages.isCategory = 0'

#graph.run(commandToRun)

Wall time: 0 ns


##### Marking categories to drop and dropping

In [23]:
%%time
# Add label "Include" to all pages (Wall time: 24min 41s)
# NOTE - Not used

commandToRun = "MATCH (pages:Page) \
                SET pages:Include"

#graph.run(commandToRun)

Wall time: 0 ns


In [24]:
%%time
# Remove label "Include" from pages included in list of categories to exclude (Wall time: 1min 33s)
# NOTE - Could remove label Include from all nodes

commandToRun = "MATCH (pages:Category:Page) \
                WHERE \
                pages.title STARTS WITH 'Wikipedia_' \
                OR pages.title STARTS WITH '1' \
                OR pages.title STARTS WITH '2' \
                OR pages.title STARTS WITH '3' \
                OR pages.title STARTS WITH '4' \
                OR pages.title STARTS WITH '5' \
                OR pages.title STARTS WITH '6' \
                OR pages.title STARTS WITH '7' \
                OR pages.title STARTS WITH '8' \
                OR pages.title STARTS WITH '9' \
                OR pages.title STARTS WITH '0' \
                OR pages.title STARTS WITH 'List_of' \
                OR pages.title STARTS WITH 'All_articles' \
                OR pages.title STARTS WITH 'Articles_' \
                OR pages.title CONTAINS 'by_year' \
                OR pages.title CONTAINS 'of_the_year' \
                OR pages.title CONTAINS '_in_' \
                REMOVE pages:Include"

#graph.run(commandToRun)

Wall time: 0 ns


In [25]:
%%time
# Add label "Exclude" to all pages to exclude (Wall time: 1min 31s)

commandToRun = "MATCH (pages:Category:Page) \
                WHERE \
                pages.title STARTS WITH 'Wikipedia_' \
                OR pages.title STARTS WITH '1' \
                OR pages.title STARTS WITH '2' \
                OR pages.title STARTS WITH '3' \
                OR pages.title STARTS WITH '4' \
                OR pages.title STARTS WITH '5' \
                OR pages.title STARTS WITH '6' \
                OR pages.title STARTS WITH '7' \
                OR pages.title STARTS WITH '8' \
                OR pages.title STARTS WITH '9' \
                OR pages.title STARTS WITH '0' \
                OR pages.title STARTS WITH 'List_of' \
                OR pages.title STARTS WITH 'All_articles' \
                OR pages.title STARTS WITH 'Articles_' \
                OR pages.title CONTAINS 'by_year' \
                OR pages.title CONTAINS 'of_the_year' \
                OR pages.title CONTAINS '_in_' \
                SET pages:Exclude"

#graph.run(commandToRun)

Wall time: 0 ns


In [26]:
%%time
# Add label "Exclude" to all pages to exclude (Wall time: 6.69 s)

commandToRun = "MATCH (pages:Category:Page) \
                WHERE \
                pages.title CONTAINS '_categories' \
                OR pages.title = 'Webarchive_template_wayback_links' \
                SET pages:Exclude"

#graph.run(commandToRun)

Wall time: 0 ns


In [27]:
%%time
# Add label "Exclude" to all pages to exclude (Wall time: 6.69 s)

commandToRun = "MATCH (pages:Category:Page) \
                WHERE \
                pages.title = 'People_by_status' \
                SET pages:Exclude"

#graph.run(commandToRun)

Wall time: 0 ns


In [28]:
%%time
# Add label "Exclude" to all pages to exclude (Wall time:  s)

commandToRun = "MATCH (pages:Category:Page) \
                WHERE \
                pages.title = 'Categories_by_language' \
                SET pages:Exclude"

#graph.run(commandToRun)

Wall time: 0 ns


In [29]:
%%time
# DETACH and DELETE all nodes with label Exclude, iterate using APOC (Wall time: 39min 21s)
# https://neo4j.com/developer/kb/large-delete-transaction-best-practices-in-neo4j/

commandToRun = "CALL apoc.periodic.iterate('MATCH (pages:Exclude) \
                RETURN pages', \
                'DETACH DELETE pages', \
                {batchSize:1000}) \
                YIELD batches, total \
                RETURN batches, total"

#graph.run(commandToRun)


Wall time: 0 ns


In [30]:
%%time
# Add label "Exclude" to all pages to exclude (Wall time: 6.69 s)

commandToRun = "MATCH (pages:Category:Page) \
                WHERE \
                pages.title = 'Sources' \
                SET pages:Exclude"

#graph.run(commandToRun)

Wall time: 0 ns


In [31]:
%%time
# DETACH and DELETE all nodes with label Exclude, iterate using APOC (Wall time: 39min 21s)
# https://neo4j.com/developer/kb/large-delete-transaction-best-practices-in-neo4j/

commandToRun = "CALL apoc.periodic.iterate('MATCH (pages:Exclude) \
                RETURN pages', \
                'DETACH DELETE pages', \
                {batchSize:1000}) \
                YIELD batches, total \
                RETURN batches, total"

#graph.run(commandToRun)


Wall time: 0 ns


##### Changing relations to optimize category tree

In [32]:
%%time
# HELPER COMMAND FOR INFO
# Replace (People) - [BELONGS_TO] -> (MTC) relation with [BELONGS_TO_CUT]
# MTC wikipedia-id: 7345184
commandToRun = 'MATCH (mtc:Page {id: 7345184}) <-[r]- (linker:Page {id: 3260154}) \
                RETURN r'
#graph.run(commandToRun).data()

Wall time: 0 ns


In [33]:
%%time
# Add [BELONGS_TO_CUT] relationship between MTC and People
# MTC wikipedia-id: 7345184, People wikipedia-id: 691008
commandToRun = 'MATCH (MTC:Page {id: 7345184}), (People:Page {id: 691008}) \
                CREATE (MTC) <-[:BELONGS_TO_CUT]- (People)'
# graph.run(commandToRun).data()

Wall time: 0 ns


In [34]:
%%time
# Remove [BELONGS_TO] relationship between MTC and People
# MTC wikipedia-id: 7345184, People wikipedia-id: 691008
commandToRun = 'MATCH (MTC:Page {id: 7345184}) <-[r:BELONGS_TO]- (People:Page {id: 691008}) \
                DELETE r'
# graph.run(commandToRun).data()

Wall time: 0 ns


In [35]:
%%time
# Add [BELONGS_TO_CUT] relationship between MTC and World
# MTC wikipedia-id: 7345184, World wikipedia-id: 3260154
commandToRun = 'MATCH (MTC:Page {id: 7345184}), (World:Page {id: 3260154}) \
                CREATE (MTC) <-[:BELONGS_TO_CUT]- (World)'
#graph.run(commandToRun).data()

Wall time: 0 ns


In [36]:
%%time
# Remove [BELONGS_TO] relationship between MTC and World
# MTC wikipedia-id: 7345184, World wikipedia-id: 3260154
commandToRun = 'MATCH (MTC:Page {id: 7345184}) <-[r:BELONGS_TO]- (World:Page {id: 3260154}) \
                DELETE r'
#graph.run(commandToRun).data()

Wall time: 0 ns


In [37]:
%%time
# Add [BELONGS_TO] relationship between MTC and Entertainment
# MTC wikipedia-id: 7345184, Entertainment wikipedia-id: 693016
commandToRun = 'MATCH (MTC:Page {id: 7345184}), (Entertainment:Page {id: 693016}) \
                CREATE (MTC) <-[:BELONGS_TO]- (Entertainment)'
#graph.run(commandToRun).data()

Wall time: 0 ns


## Step B
##### Calculate similarity between two categories (Note: not an article and a category)

## Step A
##### Article - Category relationship strength (TBD)

In [38]:
'''
We calculate the article classification strength Ac;a of
an article a for a given category c as: Ac;a = 1+ Aa intersection Ac, where
Aa is the set of articles linking to, or being linked to by, article a;
and Ac is the set of all articles classified under category c (the classification
of article a to category c is the initial number 1 on the
equation’s right-hand side).
'''

'\nWe calculate the article classification strength Ac;a of\nan article a for a given category c as: Ac;a = 1+ Aa intersection Ac, where\nAa is the set of articles linking to, or being linked to by, article a;\nand Ac is the set of all articles classified under category c (the classification\nof article a to category c is the initial number 1 on the\nequation’s right-hand side).\n'

In [45]:
%%time
ArticleTitle = "Na_Klang_District"
chosenPathArticleToMTC(articleByTitle(ArticleTitle).iloc[0,1])

Wall time: 49.8 s


,pages.title,pages.id,intersection,union,jaccard,depth,similarityBAC,mostSimilar,comment
0,Isan_geography_stubs,10006151,NaN,NaN,NaN,5,NaN,NaN,NaN
1,Thailand_geography_stubs,1440492,0.0,395.0,0.0,4,4.0,True,Lowest similarityBAC
2,Asia_geography_stubs,1476648,0.0,214.0,0.0,3,3.0,True,Lowest similarityBAC
3,Geography_stubs,931823,0.0,162.0,0.0,2,2.0,True,Lowest similarityBAC
4,Geography,693800,0.0,140.0,0.0,1,1.0,True,Lowest similarityBAC
5,Main_topic_classifications,7345184,0.0,118.0,0.0,0,0.0,True,Lowest similarityBAC


In [46]:
%%time
ArticleTitle = "Brad_Pitt"
parentCategories(articleByTitle(ArticleTitle).iloc[0,1])

Wall time: 25.9 ms


,pages.title,pages.id
0,American_male_film_actors,38422423
1,American_male_voice_actors,40168018
2,American_male_television_actors,38422437
3,Featured_articles,8966941
4,Living_people,3782398
5,Male_actors_from_Oklahoma,40437871
6,Producers_who_won_the_Best_Picture_Academy_Award,17659363
7,Male_actors_from_Missouri,40438093
8,American_film_producers,3082434
9,University_of_Missouri_School_of_Journalism_alumni,56567836


In [41]:
#categoryByTitle("Vehicles")

In [42]:
#parentCategories(5523119)

In [43]:
#childPages(4892515)

In [44]:
#childPages(722196)